In [ ]:
from component.CalculoIndiceUnidadeFederativaComponent import CalculoIndiceUnidadeFederativaComponent
from model.constants.ParametrosConstantes import ParametrosConstantes
from service.UnidadeFederativaService import UnidadeFederativaService
from util.BancoDadosUtil import BancoDadosUtil
from util.DataFrameUtil import DataFrameUtil
from util.LoggerUtil import LoggerUtil
from util.WarningUtil import WarningUtil

from pandas import DataFrame
from time import sleep

In [ ]:
# Configurações iniciais

WarningUtil.ignorar_warning(categoria=DeprecationWarning)
WarningUtil.ignorar_warning(categoria=UserWarning)

log = LoggerUtil.configurar_logger(arquivo_log=ParametrosConstantes.CAMINHO_LOG_CALCULO_INDICE_UNIDADE_FEDERATIVA)

In [ ]:
calculo_indice_component = CalculoIndiceUnidadeFederativaComponent()
unidade_federativa_service = UnidadeFederativaService()

In [ ]:
while True:
    log.info(msg="Iniciando o cálculo do indíce de 15 minutos das unidades federativas.")
    conexao_bd = BancoDadosUtil.estabelecer_conexao_banco()

    if calculo_indice_component.verificar_nao_existencia_registros_pendentes(conexao_bd):
        BancoDadosUtil.encerrar_conexao_banco(conexao_bd)
        break

    df_municipio = unidade_federativa_service.buscar_registros_pendentes_calculo_indice_15min(conexao_bd)

    log.info(msg="Calculando o índice de 15 minutos para as unidades federativas.")

    df_resultado = DataFrameUtil.processar_dataframe_dask(
        df=df_municipio,
        funcao=calculo_indice_component.processar_particao_dask,
        meta=DataFrame(columns=["codigo_unidade_federativa", "lista_dict_historico_erro", "status"]),
        qtde_particoes=1,
        conexao_bd=conexao_bd
    )

    calculo_indice_component.persistir_resultado(df_resultado, conexao_bd)

    BancoDadosUtil.encerrar_conexao_banco(conexao_bd)
    sleep(10)